In [1]:
import pandas as pd
from statsmodels.stats.multitest import multipletests
import numpy as np

In [2]:
uncorrected_df = pd.read_csv("full_10k_permutation.csv")

In [3]:
uncorrected_df

,Cancer,A1BG,A1CF,A2M,A2ML1,A4GALT,AAAS,AACS,AADAC,AADAT,...,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,ccrcc,0.052810,0.000838,0.029716,NaN,0.337474,0.728086,6.413643e-02,0.350422,0.057700,...,NaN,1.506673e-01,3.202242e-02,1.603670e-01,NaN,0.033585,5.243498e-01,8.419726e-02,3.727352e-01,0.202787
1,en,0.781720,NaN,0.911887,1.389663e-02,0.332572,0.005615,6.073417e-09,NaN,0.309108,...,0.601641,7.465966e-01,7.258794e-06,5.565692e-09,NaN,0.285104,8.265039e-02,5.077769e-01,2.323887e-01,0.196638
2,luad,0.758399,NaN,0.010615,NaN,NaN,0.139531,1.567981e-36,0.022448,0.017033,...,NaN,4.054898e-10,5.038784e-20,1.490761e-05,NaN,0.577845,1.056011e-10,4.521974e-08,2.750287e-15,0.000013
3,hnscc,0.996778,0.776459,0.228337,5.558182e-04,0.040184,0.014557,8.203827e-01,0.000280,NaN,...,NaN,6.634511e-04,1.484058e-01,1.437383e-03,NaN,0.002749,8.339845e-04,8.879081e-01,1.516343e-01,0.762835
4,lscc,0.137994,NaN,0.097044,2.033188e-22,0.154022,0.008081,7.086254e-02,0.456664,NaN,...,0.001427,1.917384e-13,7.815859e-51,2.845191e-33,NaN,0.011741,7.526734e-03,3.401658e-09,1.794113e-06,0.018557


In [4]:
gene_list = uncorrected_df.columns[1:]

In [5]:
def get_non_nan(input_list):
    non_nan_vals = []
    non_nan_indices = []
    for x in range(len(input_list)):
        if str(input_list[x]) != 'nan':
            non_nan_vals.append(input_list[x])
            non_nan_indices.append(x)
    return non_nan_vals, non_nan_indices

In [6]:
ccrcc_df,ccrcc_indices = get_non_nan(uncorrected_df.iloc[0][1:])
en_df,en_indices = get_non_nan(uncorrected_df.iloc[1][1:])
luad_df,luad_indices = get_non_nan(uncorrected_df.iloc[2][1:])
hnscc_df,hnscc_indices = get_non_nan(uncorrected_df.iloc[3][1:])
lscc_df,lscc_indices = get_non_nan(uncorrected_df.iloc[4][1:])

In [7]:
ccrcc_corrected = multipletests(ccrcc_df,method="fdr_bh")[1]
en_corrected = multipletests(en_df,method="fdr_bh")[1]
luad_corrected = multipletests(luad_df,method="fdr_bh")[1]
hnscc_corrected = multipletests(hnscc_df,method="fdr_bh")[1]
lscc_corrected = multipletests(lscc_df,method="fdr_bh")[1]

In [8]:
def make_corrected_with_nan(corrected_list,non_nan_indices,gene_list):
    return_list = []
    corrected_list_counter = 0
    for x in range(len(gene_list)):
        if x in non_nan_indices:
            return_list.append(corrected_list[corrected_list_counter])
            corrected_list_counter += 1
        else:
            return_list.append(np.nan)
    return return_list

In [9]:
ccrcc_corrected_with_nan = make_corrected_with_nan(ccrcc_corrected,ccrcc_indices,gene_list)
en_corrected_with_nan = make_corrected_with_nan(en_corrected,en_indices,gene_list)
luad_corrected_with_nan = make_corrected_with_nan(luad_corrected,luad_indices,gene_list)
hnscc_corrected_with_nan = make_corrected_with_nan(hnscc_corrected,hnscc_indices,gene_list)
lscc_corrected_with_nan = make_corrected_with_nan(lscc_corrected,lscc_indices,gene_list)

In [10]:
df_list = [ccrcc_corrected_with_nan,en_corrected_with_nan,luad_corrected_with_nan,hnscc_corrected_with_nan,lscc_corrected_with_nan]

In [11]:
total_corrected_df = pd.DataFrame(df_list, columns =gene_list, dtype = float)

In [12]:
total_corrected_df.insert(0,"Cancer",["ccrcc","en","luad","hnscc","lscc"],True)

In [13]:
total_corrected_df

,Cancer,A1BG,A1CF,A2M,A2ML1,A4GALT,AAAS,AACS,AADAC,AADAT,...,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,ccrcc,0.146442,0.004995,0.095512,NaN,0.532265,0.851988,1.691592e-01,0.545156,0.156638,...,NaN,3.098461e-01,1.010117e-01,3.231642e-01,NaN,0.104305,7.044145e-01,2.069261e-01,5.672603e-01,0.378694
1,en,0.883318,NaN,0.953553,5.628600e-02,0.543085,0.027277,1.438315e-07,NaN,0.517563,...,0.769929,8.608802e-01,8.894419e-05,1.330422e-07,NaN,0.491506,2.125946e-01,6.985013e-01,4.283120e-01,0.383779
2,luad,0.801031,NaN,0.019206,NaN,NaN,0.191032,5.371135e-35,0.037629,0.029363,...,NaN,2.594465e-09,7.619459e-19,4.915995e-05,NaN,0.641317,7.220021e-10,2.219300e-07,2.915594e-14,0.000043
3,hnscc,0.998985,0.862365,0.381438,3.371964e-03,0.103999,0.046917,8.900750e-01,0.001904,NaN,...,NaN,3.904316e-03,2.797887e-01,7.448440e-03,NaN,0.012482,4.719306e-03,9.340559e-01,2.842001e-01,0.854269
4,lscc,0.183336,NaN,0.133999,2.266058e-21,0.202066,0.014338,1.012628e-01,0.522300,NaN,...,0.002938,1.262976e-12,2.357531e-49,5.104002e-32,NaN,0.020118,1.345127e-02,1.587683e-08,6.037113e-06,0.030463


In [14]:
# total_corrected_df.to_csv("full_10k_permutation_corrected.csv",index=False)